In [44]:
import pandas as pd
import numpy as np

In [45]:
data = pd.read_csv('../Data/dataset_1000_minimum_reviews.csv', index_col='Unnamed: 0')
data

,user_id,movie_id,rating,genre,gender,age,occupation,time,binary_rating
0,3669,1197,4,Action|Adventure|Comedy|Romance,M,25,14,0,0
1,3621,780,2,Action|Sci-Fi|War,M,18,4,1,0
2,4917,1234,3,Comedy|Crime,M,45,0,2,0
3,1147,1544,2,Action|Adventure|Sci-Fi|Thriller,M,25,20,3,0
4,262,3175,5,Adventure|Comedy|Sci-Fi,F,25,1,4,1
...,...,...,...,...,...,...,...,...,...
284267,2240,1544,5,Action|Adventure|Sci-Fi|Thriller,M,45,12,284267,1
284268,5550,1089,4,Crime|Thriller,M,35,15,284268,0
284269,5988,1197,4,Action|Adventure|Comedy|Romance,M,25,15,284269,0
284270,3764,923,5,Drama,M,25,1,284270,1


In [78]:
# As usual, we only keep our features of interess
df = data[['time', 'movie_id', 'user_id', 'rating']].copy()
df['rating'] = (df['rating']-1)/4
df.head()

,time,movie_id,user_id,rating
0,0,1197,3669,0.75
1,1,780,3621,0.25
2,2,1234,4917,0.50
3,3,1544,1147,0.25
4,4,3175,262,1.00


In [96]:
def EXP_3(r_t, w, eta, arms, N):
    W_t = np.sum(w)
    Probas = []
    for a in range(N):
        P_a = (1-eta)*w[a]/W_t + eta/N
        Probas.append(P_a)
    A_t = np.random.choice(np.arange(N) , p = Probas)
    for a in range(N):
        if A_t == a:
            l_hat = ( 1 - r_t )/Probas[a]
        else:
            l_hat = 0
        w = w*np.exp( -eta * l_hat/N )
    return(arms[A_t],w)

In [97]:
def policy_evaluator_EXP3(dataframe, eta):
    # We get the list of the arms (the movies)
    arms = dataframe['movie_id'].unique().tolist()
    N = len(arms)
    w = np.ones(N)
    # We initialize an empty history
    history = pd.DataFrame([], columns = dataframe.columns)
    # We stock the payoffs in a list
    payoffs = []
    for t in range(len(dataframe)):
        # We check our t-th row
        t_event = dataframe[t : t + 1]
        a_t , w = EXP_3(dataframe['rating'].iloc[t], w, eta, arms, N)
        # If the movie recommended matches the movie of our dataframe, we update our history and our payoffs
        if a_t == t_event['movie_id'].iloc[0] :
            history.loc[len(history)] = t_event.iloc[0].to_list()
            payoffs.append(t_event['binary_rating'].iloc[0])
    return payoffs